This notebook focuses on generating summaries by using BART large CNN model (https://huggingface.co/facebook/bart-large-cnn) from the impact sentences extracted earlier. It produces two distinct summaries: one highlighting structural impacts and the other detailing community impacts. The summarized results will be available in .txt format.

Please ensure to paste the input_path, which is the location of the output of the 01_input.ipynb file, at the start of this notebook. This step is the only requirement to load all necessary information for the execution of the code.

Recommended Google Colab Runtime Type: A100 GPU (preferred), as this notebook involves running machine learning models.

In [ ]:
# Specify the directory path where the output of the input file 01_input.ipynb was saved
input_path = "/content/drive/My Drive/ImpactDataMining/Turkiye_Earthquake/01_Input"

All the below sections automatically retrieve data from the 01_input.ipynb file, as well as results from previous notebooks in this series. The code is designed to run using this information, so no further edits are required beyond this point.

In [ ]:
!pip install transformers
import os
import json
import torch
import math
import re

from google.colab import drive
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration

In [ ]:
import time

start_time = time.time()

In [ ]:
def current_path():
  print("Current working directory")
  print(os.getcwd())
  print()

current_path()
drive.mount('/content/drive')
os.chdir(input_path)
current_path()

Current working directory
/content

Mounted at /content/drive
Current working directory
/content/drive/My Drive/ImpactDataMining/Turkiye_Earthquake/01_Input



In [ ]:
with open('0_input.json', 'r') as file:
    data = json.load(file)
    output_path = data['output_path']
    overlap_tokens = data['overlap_tokens']

In [ ]:
os.makedirs(output_path, exist_ok=True)
os.chdir(output_path)
current_path()

Current working directory
/content/drive/My Drive/ImpactDataMining/Turkiye_Earthquake/03_Output



In [ ]:
with open('2b_output.json', 'r') as file:
    data = json.load(file)
    sent_nested = data['sent_nested']
    sent_struct_nested = data['sent_struct_nested']
    sent_comm_nested = data['sent_comm_nested']

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def summarize_with_bart(chunk, mp, ml=None):
  if len(chunk.split()) > 30:
    inputs = tokenizer(chunk, max_length=1024, return_tensors='pt', truncation=True)
    inputs.to(device)

    inputs_len = inputs['input_ids'].shape[1]
    max_len = inputs_len
    if ml is None:
        min_len = int(mp * inputs_len)
    else:
        min_len = ml
        if min_len > inputs_len:
            min_len = int(mp * inputs_len)

    inputs_len = inputs['input_ids'].shape[1]
    summary_ids = model.generate(inputs['input_ids'], max_length=max_len, min_length=min_len,
                  do_sample=False, early_stopping=True, num_beams=4)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
  else:
    summary = chunk

  return summary

In [ ]:
def summary_chunk(text):
  result = []
  for n in text:
    if len(n) > 1:
      para = ' '.join([sent for sent in n])
      summary = summarize_with_bart(para, .8, 30)
      result.append(summary)
    elif len(n) == 1 and len(n[0].split()) > 30:
      para = n[0]
      summary = summarize_with_bart(para, .8, 30)
      result.append(summary)
    else:
      result.append(n[0])
  return result

In [ ]:
def sliding_window(text, max_tokens=1024, overlap_tokens=200):
  para = text
  para_len = [len(n.split()) for n in para]
  chunks_len = []; chunks = []

  running_sum_len = 0; running_sum_text = '';
  overlap = []; overlap_1 = [];

  for count, text in zip(para_len, para):
    if running_sum_len + count > max_tokens:
      chunks_len.append(running_sum_len)
      chunks.append(running_sum_text)

      overlap_count = 0; overlap_text = '';
      for prev_elem, prev_text in zip(reversed(overlap), reversed(overlap_1)):
        if overlap_count + prev_elem < overlap_tokens:
          overlap_count = overlap_count + prev_elem
          overlap_text = ' '.join([overlap_text, prev_text])
        else:
          break

      running_sum_len = overlap_count + count
      running_sum_text = ' '.join([overlap_text, text])

      overlap = []; overlap_1 = []
    else:
      running_sum_len = running_sum_len + count
      running_sum_text = ' '.join([running_sum_text, text])

      overlap.append(count)
      overlap_1.append(text)

  if running_sum_len:
    chunks_len.append(running_sum_len)
    chunks.append(running_sum_text)
  return chunks

def hierarchical_summarize(text, max_tokens=1024, overlap_tokens=200):
    chunks = sliding_window(text, max_tokens=max_tokens, overlap_tokens=overlap_tokens)
    count = 0
    summaries = []
    for chunk in chunks:
        summary = summarize_with_bart(chunk, .4)
        summaries.append(summary)
    aggregated_summary = " ".join([n for n in summaries])

    if len(aggregated_summary.split()) > max_tokens:
        return hierarchical_summarize(summaries, max_tokens=max_tokens)
    else:
        return aggregated_summary

In [ ]:
result_struct_summ = summary_chunk(sent_struct_nested)
summ_struct = hierarchical_summarize(result_struct_summ, overlap_tokens=overlap_tokens)

print('Structural impact summary:')
summ_struct

Structural impact summary:


'The Dead Sea Fault (DSF) is a major transform fault that extends from the Red Sea to the southeast of Turkey. The mechanism and location of the earthquake are consistent with the DSF and the EAF. The earthquake was followed by many aftershocks, including several larger than magnitude 6 (one with Mw 6.6). As of February 19, 2023, the number of reported completely and partially collapsed buildings was 28,362. 75,717 buildings and 306,563 dwellings were either collapsed or severely damaged. In Syria, more than 22,000. buildings were affected by the earthquakes with 2,850 of them partially/completely collapsed orseverely damaged. Almost two-thirds of residential buildings in Gaziantep city are masonry. One-third are Reinforced Concrete (RC) The lessons learned from the 1999 Kocaeli and Duzce earthquakes led to the implementation of earthquake engineering principles in the whole country. The observations from the recent Mw6.1Duzce Earthquake on November 23, 2022, showed the importance of t

In [ ]:
result_comm_summ = summary_chunk(sent_comm_nested)
summ_comm = hierarchical_summarize(result_comm_summ, overlap_tokens=overlap_tokens)

print('Community impact summary:')
summ_comm

Community impact summary:


'As of March 9, 2023, the total death toll was reported to be 45,968 people in Türkiye and around 6,000 in Syria. More than 100,000 people were reported as injured. The number of fatalities estimated by PAGER for the Mw 7.8 event is shown in Figure 1.2. For the MW 7.5 earthquake, economic losses are shown in Figures 1.3 and 1.4. Extreme Event Solutions at Verisk predicted that the economic losses and industry-insured losses due to the earthquake sequence will likely exceed $20 billion. In terms of economic loss, the ramifications could include the spoiling of food items requiring refrigeration. There is past evidence of the collapse of war-damaged residential buildings in Aleppo and consequential casualties (Reuters 2023b). There was one casualty in this historical neighborhood, which was known as a symbol of the region due to these old buildings. Hundreds of medical facilities were damaged, disrupting treatment capacities across large regions. It was delivering emergency healthcare as

In [ ]:
# Saving results to a .txt file
with open('3_structural_impact_summary.txt', 'w') as file:
    file.write(summ_struct)
with open('3_community_impact_summary.txt', 'w') as file:
    file.write(summ_comm)

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")

Execution time: 208.45172429084778 seconds
